In [2]:
print("CARGANDO LIBRERIAS NECESARIAS")


import time

import torch
import pandas as pd
import numpy as np

from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score


In [9]:
print("CARGANDO MODELO")


output_model_file = '../models/pytorch_textclasif.bin'
output_vocab_file = '../models/vocab_textclasif.bin'

# Model class must be defined somewhere
model = torch.load(output_model_file)
tokenizer = BertTokenizer.from_pretrained(output_vocab_file)


device='cpu'
#model.to('cpu')


/home/juanpablosokil/.local/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1645: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,


The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (119547, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              

In [61]:
print("CARGANDO DATOS")


df_test = pd.read_csv('../bases/2022-03.csv', sep=';', encoding='utf-8')  
df_test.shape

sentences = df_test.iloc[:, 5].values


CARGANDO DATOS


In [64]:
etiquetas=['no lo uso', 'servicio', 'funcionalidad', 'contenido', 'conveniencia', 'relacionamiento', 'otros']



predicciones= []
start_time = time.time()

for sentence in sentences:
    try:
        input_text = sentence
        input_text_tokenized = tokenizer.encode(input_text, truncation=True,padding=True,return_tensors="pt")

        prediction = model(input_text_tokenized)
        prediction_logits = prediction[0]
        prediction_probs = prediction_logits.detach().cpu().numpy()
        prediccion=etiquetas[int(prediction_probs.argmax(axis=1))]
        #print(f'The prediction probs are: {prediction_probs}')
        print("la etiqueta es :", etiquetas[int(prediction_probs.argmax(axis=1))])
        predicciones.append(prediccion)
    except:
        print("NO HAY TEXTO PARA CLASIFICAR")
        prediccion =''
        predicciones.append(prediccion)

print("El tiempo de prediccion fue de: ",(time.time() - start_time) / 60," minutos")

la etiqueta es : servicio
NO HAY TEXTO PARA CLASIFICAR
NO HAY TEXTO PARA CLASIFICAR
la etiqueta es : relacionamiento
la etiqueta es : servicio
la etiqueta es : conveniencia
la etiqueta es : funcionalidad
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : relacionamiento
la etiqueta es : servicio
la etiqueta es : funcionalidad
NO HAY TEXTO PARA CLASIFICAR
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : otros
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : funcionalidad
la etiqueta es : no lo uso
la etiqueta es : contenido
la etiqueta es : otros
la etiqueta es : servicio
la etiqueta es : contenido
la etiqueta es : servicio
la etiqueta es : conveniencia
la etiqueta es : contenido
la etiqueta es : funcionalidad
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : funcionalidad
la etiqueta es : conveniencia
NO HAY TEXTO PARA CLASIFICAR
la etiqueta es : servicio
la 

la etiqueta es : servicio
la etiqueta es : contenido
la etiqueta es : servicio
la etiqueta es : otros
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : conveniencia
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : funcionalidad
la etiqueta es : servicio
la etiqueta es : relacionamiento
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : conveniencia
la etiqueta es : funcionalidad
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : contenido
la etiqueta es : servicio
la etiqueta es : relacionamiento
la etiqueta es : servicio
la etiqueta es : contenido
la etiqueta es : funcionalidad
la etiqueta es : conveniencia
la etiqueta es : conveniencia
NO HAY TEXTO PARA CLASIFICAR
la etiqueta es : servicio
la etiqueta es : funcionalidad
NO HAY TEXTO PARA CLASIFICAR
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : contenido
la etiq

la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : conveniencia
la etiqueta es : servicio
la etiqueta es : otros
la etiqueta es : conveniencia
la etiqueta es : servicio
la etiqueta es : conveniencia
la etiqueta es : servicio
la etiqueta es : contenido
la etiqueta es : servicio
la etiqueta es : servicio
la etiqueta es : funcionalidad
la etiqueta es : conveniencia
la etiqueta es : servicio
la etiqueta es : contenido
NO HAY TEXTO PARA CLASIFICAR
la etiqueta es : servicio
la etiqueta es : relacionamiento
la etiqueta es : servicio
El tiempo de prediccion fue de:  0.8555786530176799  minutos


In [69]:
#we can save the predictions
df_test['predictions']=predicciones
print(df_test['predictions'].value_counts())
print(df_test['predictions'].value_counts(normalize=True))


servicio           323
funcionalidad       76
conveniencia        71
relacionamiento     45
contenido           38
                    35
otros               22
no lo uso            3
Name: predictions, dtype: int64
servicio           0.526917
funcionalidad      0.123980
conveniencia       0.115824
relacionamiento    0.073409
contenido          0.061990
                   0.057096
otros              0.035889
no lo uso          0.004894
Name: predictions, dtype: float64


In [71]:
df_test.to_csv("../bases/predicciones_mensuales.csv", encoding='latin1', index=False, sep=';')

Aplico el modelo